# CH11. 인터페이스: 프로토콜에서 ABC까지

* 1. 파이썬 문화에서의 인터페이스와 프로토콜
* 2. 파이썬은 시퀀스를 찾아낸다
* 3. 런타임에 프로토콜을 구현하는 멍키 패칭
* 4. 알렉스 마르텔리의 물새
* 5. ABC 상속하기
* 6. 표준 라이브러리의 ABC
* 7. AB의 정의와 사용
* 8. Tombola 서브클래스 테스트 방법
* 9. register()의 실제 용법
* 10. 오리처럼 행동할 수 있는 거위

## 11.1 파이썬 문화에서의 인터페이스와 프로토콜
* 덕 타이핑 & 프로토콜 : 파이썬과 같은 동적 자료형을 제공하는 언어에서 다형성을 제공하는 비공식 인터페이스
* 파이썬에는 interface 키워드가 없지만, 클랙스가 상속하거나 구현한 공개 속성(메서드나 데이터 속성)들의 집합이 인터페이스이다.
  * ex) `__getitem__()`, `__add__()`
  * 보호된 속성이나 비공개 속성은 인터페이스에 속하지 않음

In [2]:
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    # 이후 메서드 생략

In [3]:
for i in Vector2d(4, 5):
    print(i)

4.0
5.0


In [4]:
## 읽기 전용 프로퍼티로 변경 => 하지만 인터페이스의 핵심 부분은 변한게 없음
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    # 이후 메서드 생략

In [5]:
v1 = Vector2d(4, 5)
print(v1.x)
v1.x = 6

4.0


AttributeError: can't set attribute

* 인터페이스 : 시스템에서 어떤 역할을 할 수 있게 해주는 객체의 공개 메서드의 일부
* 일부 언어에서는 어떤 역할을 완수하기 위한 메서드 집합으로서의 인터페이스를 프로토콜이라 부름. 프로토콜은 상속과 무관
* 프로토콜은 인터페이스이지만 비공식적이다. 문서와 관례에 따라 정의되지만 강제할 수 없음.

## 11.2 파이썬은 시퀀스를 찾아낸다
* 시퀀스 프로토콜은 파이썬에서 가장 핵심적인 인터페이스

abc.Sequence 관계 (p395)

In [ ]:
# abc.Sequence 를 상속하지 않고 __getitem__ 메서드만 구현
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

f = Foo()
print(f[1])

# __iter__() 가 구현되어 있지 않지만 가능!
for i in f:
    print(i)

# __contains__() 가 구현되어 있지 않지만 가능!
print(20 in f)
print(15 in f)

1장의 예제도 같은 이유로, 상속하지 않았지만 동작

In [38]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

섞을수 없다는 단점.. 아래에서 해결

## 11.3 런타임에 프로토콜을 구현하는 멍키 패칭


In [37]:
from random import shuffle
l = list(range(10))
shuffle(l)
print(l)

[4, 3, 8, 9, 5, 1, 2, 6, 0, 7]


In [39]:
from random import shuffle

deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

가변시퀀스는 `__setitem__()` 메서드도 지원해야 함
* 멍키 패칭

In [41]:
def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card
shuffle(deck) # deck 을 새로 생성하지 않았는데도 됨!? => 프로토콜은 동적이다!
print(deck[:5])

[Card(rank='J', suit='spades'), Card(rank='A', suit='clubs'), Card(rank='10', suit='clubs'), Card(rank='Q', suit='clubs'), Card(rank='3', suit='spades')]


In [43]:
FrenchDeck.__len__

<function __main__.FrenchDeck.__len__(self)>

In [45]:
FrenchDeck.__setitem__

<function __main__.set_card(deck, position, card)>

In [50]:
FrenchDeck.suits = ['1', '2', '3', '4']

In [52]:
deck.suits

['1', '2', '3', '4']

In [53]:
print(deck[:5])

[Card(rank='J', suit='spades'), Card(rank='A', suit='clubs'), Card(rank='10', suit='clubs'), Card(rank='Q', suit='clubs'), Card(rank='3', suit='spades')]


## 11.4 알렉스 마르텔리의 물새
* 고전적인 객체 지향 패턴

In [ ]:
class Artist:
    def draw(self): ... # 그림 그리기

class Gunslinger:
    def draw(self): ... # 총을 뽑는 행위

class Lottery:
    def draw(self): ... # 복권 추첨

뭔소리인지.. 표현학.. 분기학..

draw 라고 다 대등한 개념이라고 볼 수 없다.

고전적인 덕 타이핑을 구스 타이핑으로 보완

* 구스 타이핑 : cls 가 추상 베이스 클래스인 경우, isinstance(obj, cls) 를 써도 좋다는 의미

In [54]:
class Struggle:
    def __len__(self):
        return 23

from collections import abc
print(isinstance(Struggle(), abc.Sized))

True


`__len__()` 만 구현하면 abc.Sized 의 서브클래스로 인식

단, 이런 특별 메서드를 구현할 때에는 구문과 의미를 적절히 지켜야 함.

* 언제나 ABC 개념을 실현하는 클래스를 구현할때에는 ABC 클래스를 상속하거나 등록.
* 그리고 isinstance() 를 통해 검사
* 배포용 코드에서는 절대로 ABC 직접 구현하지 않기.
  * 상속은 개발자의 의도를 명확히 나타낸다. 가상 서브 클래스 등록도 마찬가지

* isinstance() 를 너무 많이 사용하는 것은 안좋음. 다형성을 이용하여 인터프리터가 적절히 호출할 수 있도록 구현

In [ ]:
field_names = "a,b,c,d"

# 덕 타이핑으로 구현하는게 낫다!
try:
    field_names = field_names.replace(',', ' ').split()
except AttributeError:
    print("attr error")
    pass
field_names = tuple(field_names)
print(field_names)

In [ ]:
field_names = [1,2,3,4]

try:
    field_names = field_names.replace(',', ' ').split()
except AttributeError:
    print("attr error")
    pass
field_names = tuple(field_names)
print(field_names)

직접 ABC 만들기 보다는 기존 ABC 를 제대로 사용하는 것만으로도 큰 혜택을 볼수 있다.

## 11.5 ABC 상속하기


In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.abc.MutableSequence): # collections 에서 바로 불러오는것은 deprecated 됨
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
    
    def __delitem__(self, position):
        del self._cards[position]
    
    def insert(self, position, value):
        self._cards.insert(position, value)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """

In [3]:
# 구현을 안하면?
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.abc.MutableSequence): # collections 에서 바로 불러오는것은 deprecated 됨
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
    
#     def __delitem__(self, position):
#         del self._cards[position]
    
    def insert(self, position, value):
        self._cards.insert(position, value)

In [4]:
# 모듈을 로딩하거나 컴파일 할때는 문제 없음. 실제 객체 생성시 추상 메서드 구현 여부 확인
FrenchDeck2()

TypeError: Can't instantiate abstract class FrenchDeck2 with abstract methods __delitem__

ABC의 구상 메서드는 클래스 공개 인터페이스만 이용하여 구현하므로 내부 구조 몰라도 작동

## 11.6 표준 라이브러리의 ABC
* 대부분 collections.abc 에 정의되어 있음. (특별한 경우 아니고 abc.ABC 를 임포트할 일 없음)

P408 그림 참고

숫자탑 (Number > Complex > Real > Rational > Integral)

## 11.7 ABC의 정의와 사용

In [5]:
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """iterable의 항목들을 추가한다."""
    
    @abc.abstractmethod
    def pick(self):
        """무작위로 항목을 하나 제거하고 반환한다.
        객체가 비어 있을 때 이 메서드를 실행하면 `LookupError`가 발생한다.
        """
    
    def loaded(self):
        """최소 한 개의 항목이 있으면 True를, 아니면 False를 반환한다."""
        return bool(self.inspect())
    
    def inspect(self):
        """현재 안에 있는 항목들로 구성된 정렬된 튜플을 반환한다."""
        items =[]
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sored(items))
    

In [6]:
class Fake(Tombola):
    def pick(self):
        return 13

Fake

__main__.Fake

In [7]:
f = Fake() # 추상 메서드를 구현하지 않았기 때문에 객체 생성 시점에서 오류 발생

TypeError: Can't instantiate abstract class Fake with abstract methods load

데커레이터는 중첩 가능하나 abstractmethod 를 제일 먼저 적용해야 한다!

In [9]:
class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod
    def an_abstract_classmethod(cls):
        pass

BingoCage

In [16]:
import random

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
    
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self): # return 하는게 맞지 않나..
        self.pick()

In [21]:
bingo_cage = BingoCage([1, 2, 3])
print(bingo_cage())
print(bingo_cage())
print(bingo_cage())
print(bingo_cage())

None
None
None


LookupError: pick from empty BingoCage

LotteryBlower -> 구상 메서드를 좀더 효율적으로 구성

In [24]:
import random

class LotteryBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable)
    
    def load(self, iterable):
        self._balls.extend(iterable)
    
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LotteryBlower') ## BingoCage 는 아닌거 같다
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))

In [27]:
lottery_blower = LotteryBlower([1, 2, 3])
print(lottery_blower.pick())
print(lottery_blower.load(range(3)))
print(lottery_blower.inspect())


1
None
(0, 1, 2, 2, 3)


구스 타이핑의 본질적인 기능은 어떤 클래스가 ABC 를 상속하지 않더라도 가상 서브 클래스로 등록할 수 있다는 것이다.

만약 제대로 인터페이스를 구현하지 않으면 런타임에 예외 발생.

In [28]:
from random import randrange

@Tombola.register
class TomboList(list):
    
    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
    
    load = list.extend ##???
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))
    
# Tombola.register(TomboList)
    

In [29]:
issubclass(TomboList, Tombola)

True

In [30]:
issubclass(Tombola, TomboList)

False

In [31]:
t = TomboList(range(100))
print(isinstance(t, Tombola))
print(isinstance(t, TomboList))

True
True


In [32]:
TomboList.__mro__

(__main__.TomboList, list, object)

메서드를 검색할 순서대로 자신 클래스와 슈퍼클래스들을 나열

등록된 클래스는 없다

## 11.8 Tombola 서브클래스 테스트 방법

클래스 계층구조를 조사할 수 있는 두 가지 클래스 속성

* `__subclasses__()` 클래스의 바로 아래 서브클래스의 리스트를 반환하는 메서드, 리스트에 가상 서브클래스는 없음
* `_abc_registry` 추상 클래스에 등록된 가상 서브클래스에 대한 약한 참조를 담고 있는 WeakSet

In [55]:
!python tombola_runner.py

In [ ]:
????

In [56]:
import bingo, lotto, tombolist, drum

ModuleNotFoundError: No module named 'bingo'

## 11.9 register()의 실제 용법

데커레이터를 사용할 수 있음에도 다음과 같이 등록된 경우가 많다

In [33]:
Sequence.register(tuple)
Sequence.register(str)
Sequence.register(range)
Sequence.register(memoryview)

NameError: name 'Sequence' is not defined

https://hg.python.org/cpython/file/3.4/Lib/_collections_abc.py

## 11.10 오리처럼 행동할 수 있는 거위
* 등록하지 않고도 가상 서브 클래스로 인식할 수 있다

In [35]:
class Struggle:
    def __len__(self):
        return 23

from collections import abc

print(isinstance(Struggle(), abc.Sized))
print(issubclass(Struggle, abc.Sized))

True
True


`__subclasshook__()` 을 구현하면 가능하다!

In [36]:
class Sized(metaclass=ABCMeta):
    
    __slots__ = ()
    
    @abstractmethod
    def __len__(self):
        return 0
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized: # 이 구문이 이해가 안된다..
            if any("__len__" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented

NameError: name 'ABCMeta' is not defined

그러나 이방법은 좋은 방법은 아니다..